# Plotting Mekong Development Map by Choropleth Maps

>A choropleth map is a type of thematic map in which areas are shaded or patterned in proportion to a statistical variable that represents an aggregate summary of a geographic characteristic within each area, such as population density or per-capita income.

## Terminologies
---

## [Plotly](https://plotly.com/)
Plotly is a technical computing company that develops online data analytics and visualization tools. Plotly provides online graphing, analytics, and statistics tools for individuals and collaboration, as well as scientific graphing libraries for Python, R, MATLAB, Perl, Julia, Arduino, and REST.

### [plotly.py](https://plotly.com/python/)
An interactive, open-source, and browser-based graphing library for Python.

### [GeoJSON](https://geojson.org/)
An open standard format designed for representing simple geographical features, along with their non-spatial attributes.

### [Mapbox](https://mapbox.com/)
An open source mapping platform for custom designed maps.

In [1]:
import json
import numpy as np
import pandas as pd
import plotly.express as px

from codes.plot import *

In [ ]:
import plotly.io as pio
import os

if os.environ.get("CI") == "true":
  pass
else:
  pio.renderers.default = 'browser'

In [2]:
# import the Vietnam map by provinces data
vietnam_geo = json.load(open("geodata/aquaculture_production_of_vietnam.geojson", "r"))

# # Convert map properties/
# state_id_map = {}
# for feature in vietnam_geo["features"]:
#     feature["id"] = feature["properties"]["Code"]
#     state_id_map[feature["properties"]["Name"]] = feature["id"]

## 1. The Aquaculture Production of Vietnam from 2013 to 2018 by Provinces

Published by: Open Development Vietnam
The data provides information on Vietnam's aquaculture production from 2013 to 2018. The aquaculture in Vietnam includes: farmed fish production, farmed shrimp production and other aquatic products. Aquaculture production is divided by province and city.

In [3]:
# Import aquaculture_production csv
df = pd.read_csv("geodata/tsnt_2013__2018_en.csv")
years = ['2013','2014','2015','2016','2017','2018']
for i, y in enumerate(years):
    col = 'ProdScale'+ y
    df[col] = np.log10(df[y])

# Convert wide to long format
tsnt = df.drop(years, axis=1)
final_tsnt = pd.wide_to_long(tsnt,'ProdScale', i=['Provincial name','Provincial code'], j="year")
final_tsnt.reset_index(inplace=True)

In [4]:
fig = px.choropleth(
    final_tsnt,
    locations ="Provincial code",
    animation_frame = "year",
    geojson = vietnam_geo,
    color ="ProdScale",
    hover_name = "Provincial name",
    title ="The Aquaculture Production of Vietnam from 2013 to 2018 by Provinces",
)
fig.update_geos(fitbounds ="locations", visible=False)
fig.show()

## 2. Proportion of poor households by region in Vietnam from 1998 to 2016

The dataset includes le pourcentage of poor households by region in Vietnam from 1998 to 2016. The standard of poor households for this period based on the average income per person per month of households is updated according to the consumer price index as follows: In 2010, VND 400,000 for rural areas and VND 500,000 for urban areas; Similarly, in 2013 it was VND 570,000 and VND 710,000; in 2014, VND 605,000 dong and VND 750,000; in 2015, there were VND 615,000 and VND 760,000 dong; In 2016, VND 630,000 and VND 780,000 respectively.

In [5]:
# import the Vietnam map by region data
vietnamregion_geo = json.load(open("geodata/poverty_rate_1998_2016.geojson", "r", encoding="utf8"))

# # Convert map properties/
# region_id_map = {}
# for feature in vietnamregion_geo["features"]:
#     feature["id"] = feature["properties"]["row"]
#     region_id_map[feature["properties"]["Name_EN"]] = feature["id"]

In [6]:
# Import aquaculture_production csv
poverty_rate_1998_2016 = pd.read_csv("geodata/poverty_rate_1998_2016.csv",delimiter="\t")
cols = sorted(poverty_rate_1998_2016.columns[3:], reverse=False)
poverty_rate_1998_2016.drop(['row'], axis=1, inplace = True)

for i, y in enumerate(cols):
    col = 'PovertyRate'+ y
    poverty_rate_1998_2016[col] = poverty_rate_1998_2016[y]

# Convert wide to long format
poverty = poverty_rate_1998_2016.drop(cols, axis=1)
final_poverty = pd.wide_to_long(poverty,'PovertyRate', i=['Name_EN','Name_VI'], j="year")
final_poverty.reset_index(inplace=True)

In [7]:
fig = px.choropleth(
    final_poverty,
    locations ="Name_EN",
    animation_frame = "year",
    geojson = vietnamregion_geo,
    color ="PovertyRate",
    hover_name = "Name_EN",
    title ="Proportion of poor households by region in Vietnam from 1998 to 2016",
)
fig.update_geos(fitbounds ="locations", visible=False)
fig.show()